In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import time
from IPython import display

from dedalus import public as de
from dedalus.extras import flow_tools

import logging
logger = logging.getLogger(__name__)

In [ ]:
Lx, Ly = (25,25)
nx, ny = (450, 450) 

x_basis = de.Fourier('x', nx, interval=(0, Lx), dealias=3/2)
y_basis = de.Fourier('y', ny, interval=(0, Ly), dealias=3/2)
domain = de.Domain([x_basis, y_basis], grid_dtype=np.float64)

In [ ]:
def grow_fun(A,B):
    F = (A.data)*(1-B.data)
    F[A.data < -1e-3] = 0
    return F

def grow_operator(field1,field2):
    return de.operators.GeneralFunction(
        field1.domain,
        layout = 'g',
        func = grow_fun,
        args = (field1,field2,)
    )

de.operators.parseables['grow'] = grow_operator

In [ ]:
params = np.load('Parameters.npz')
Diff = params['Diff'].flat[0]
K =  params['K'].flat[0]
g = params['g'].flat[0]

problem = de.IVP(domain, variables=['rho1','rho2','rho3','rho4','rho5','rho6','rhom'])
problem.parameters['D0'] = Diff
problem.parameters['K'] = K
problem.parameters['g0'] = g


problem.substitutions['rho_b'] = "rho1+rho2+rho3+rho4+rho5+rho6+rhom" 
problem.substitutions["Lap(A)"] = "dx(dx(A)) + dy(dy(A))"

problem.add_equation("dt(rho1)  - D0*Lap(rho1)  = g0*grow(rho1,rho_b) - K*rho1*rho2")
problem.add_equation("dt(rho2)  - D0*Lap(rho2)  = g0*grow(rho2,rho_b) - K*rho2*rho1")
problem.add_equation("dt(rho3)  - D0*Lap(rho3)  = g0*grow(rho3,rho_b) - K*rho3*rho4")
problem.add_equation("dt(rho4)  - D0*Lap(rho4)  = g0*grow(rho4,rho_b) - K*rho4*rho3")
problem.add_equation("dt(rho5)  - D0*Lap(rho5)  = g0*grow(rho5,rho_b) - K*rho5*rho6")
problem.add_equation("dt(rho6)  - D0*Lap(rho6)  = g0*grow(rho6,rho_b) - K*rho5*rho6")


problem.add_equation("dt(rhom)  = 2*K*(rho1*rho2 + rho3*rho4 +  rho5*rho6 )")

In [ ]:
ts = de.timesteppers.RK443

In [ ]:
solver =  problem.build_solver(ts)

In [ ]:
x_pos = np.linspace(0,Lx,6)[:-1]
y_pos = np.linspace(0,Ly,6)[:-1]
y_pos = y_pos + y_pos[1]/2
x_pos = x_pos + x_pos[1]/2
print(x_pos[1]-x_pos[0])
x_pos,y_pos = np.meshgrid(x_pos,y_pos)
x_pos = x_pos.ravel()# + x_pos[0,0]/2
y_pos = y_pos.ravel()# + x_pos[0,0]/2

In [ ]:
colors = [[1,1],
          [1,4],
          [2,2],
          [2,4],
          [1,1],
          [2,3],
          [4,6],
          [4,1],
          [3,6],
          [2,3],
          [4,5],
          [1,5],
          [3,5],
          [1,6],
          [4,5],
          [1,6],
          [2,4],
          [2,6],         
          [1,3],         
          [2,5],         
          [2,3],
          [3,3],
          [2,3],
          [3,3],
          [1,4]]

In [ ]:
x = domain.grid(0)
y = domain.grid(1)
rho1 = solver.state['rho1']
rho2 = solver.state['rho2']
rho3 = solver.state['rho3']
rho4 = solver.state['rho4']
rho5 = solver.state['rho5']
rho6 = solver.state['rho6']

def r(x,y,x0,y0):
    return np.sqrt((x-x0)**2+(y-y0)**2)

def blob_np(x,y,x0,y0,r0,w):
    return 0.25*(1 - np.tanh((r(x,y,x0,y0)-r0)/w))

def blob(x0,y0):
    r0 = 0.75
    w = 0.22
    return blob_np(x,y,x0,y0,r0,w)



for k in range(np.shape(colors)[0]):
    for i in range(2):
        if 1 == colors[k][i]:
            rho1['g'] = rho1['g'] + blob(x_pos[k],y_pos[k])

        if 2 == colors[k][i]:
            rho2['g'] = rho2['g'] + blob(x_pos[k],y_pos[k])

        if 3 == colors[k][i]:
            rho3['g'] = rho3['g'] + blob(x_pos[k],y_pos[k]) 

        if 4 == colors[k][i]:
            rho4['g'] = rho4['g'] + blob(x_pos[k],y_pos[k]) 
        
        if 5 == colors[k][i]:
            rho5['g'] = rho5['g'] + blob(x_pos[k],y_pos[k]) 

        if 6 == colors[k][i]:
            rho6['g'] = rho6['g'] + blob(x_pos[k],y_pos[k]) 

In [ ]:
solver.stop_sim_time = 150
dt = 0.3*Lx/nx

In [ ]:
analysis = solver.evaluator.add_file_handler('square_grid', sim_dt=2, max_writes=900)
analysis.add_task('rho1')
analysis.add_task('rho2')
analysis.add_task('rho3')
analysis.add_task('rho4')
analysis.add_task('rho5')
analysis.add_task('rho6')
analysis.add_task('rhom')

In [ ]:
logger.info('Starting loop')
start_time = time.time()
while solver.ok:
    solver.step(dt)
    if solver.iteration % 50 == 0:
        logger.info('Iteration: %i, Time: %e, dt: %e' %(solver.iteration, solver.sim_time, dt))

In [ ]:
from dedalus.tools import post
post.merge_process_files("square_grid", cleanup=True)